In [2]:
import pandas as pd

Как я уже писал ранее, данные, с которыми мы работаем, обезличены, тоесть реальные id пользователей заменены случайным числом. Анонимизация пользовательских данных - очень важная тема для Data Science. С одной стороны мы хотим знать о поведении наших пользователей как можно больше, с другой стороны нам важно соблюдать приватность наших пользователей.

Поэтому данные о поведении пользователей анонимизируют не только когда речь идет об открытых данных, но и при работе с данными внутри компании. Не заметили сколько писем вам недавно пришло об изменении политики конфеденциальности, хотите узнать почему? В комментарии приложена ссылка на очень интересный доклад об этой проблеме.

А пока что вот вам хакерская задача, за каким вымышленным id скрывается Анатолий Карпов - автор курса, данные которого мы анализируем?

Введите id Анатолия Карпова, под которым он фигурирует в данных events_data_train и submissions_data_train.

In [106]:
submissions = pd.read_csv("../data/submissions_data_train.csv")
submissions[submissions.submission_status == 'correct'].groupby('user_id').agg({'submission_status': 'count'}).sort_values(by=['submission_status'], ascending=False).head()

,submission_status
user_id,
1046,421
5552,186
17135,168
6183,142
8461,135


In [87]:
user_scores = submissions.pivot_table(index='user_id',
                                        columns='submission_status',
                                        values='step_id',
                                        aggfunc='count',
                                        fill_value=0).reset_index()
user_scores.sort_values(by='correct', ascending=False).head()

submission_status,user_id,correct,wrong
439,1046,421,513
2099,5552,186,93
6415,17135,168,177
2344,6183,142,65
3208,8461,135,42



В процессе решения одной аналитической задачи очень часто замечаешь и другие задачи, связанные с исследуемым вопросом. В свое время, работая над предсказанием оттока студентов со stepik, я многое изменил в структуре своих курсов. 

Тогда я нашел довольно интересную закономерность прохождения онлайн курсов. Слушатели онлайн курсов очень негативно реагируют на невозможность решить задачу, иными словами, если студент застрял на определенном шаге, то он, с высокой вероятность, вообще бросит курс, чем просто пропустит этот шаг и продолжит обучение. 

Давайте найдем такой стэп, используя данные о сабмитах. Для каждого пользователя найдите такой шаг, который он не смог решить, и после этого не пытался решать другие шаги. Затем найдите id шага,  который стал финальной точкой практического обучения на курсе для максимального числа пользователей. 

То есть мы исследуем следующий сценарий: человек решает стэп, не может получить правильный ответ и больше не возвращается к практическим задачам. Что это за шаг такой, который отпугнул максимальное число пользователей?

In [89]:
data = pd.read_csv("../data/submissions_data_train.csv")
data.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [98]:
df = data[data.submission_status == "wrong"]

df.loc[:, ["step_id", "submission_status"]].groupby(by="step_id").count().head()

,submission_status
step_id,
31971,6893
31972,8960
31976,5937
31977,6457
31978,16084


In [100]:
some = df.groupby(by="step_id").agg({'submission_status': 'count'})
some.head()

,submission_status
step_id,
31971,6893
31972,8960
31976,5937
31977,6457
31978,16084


In [104]:
some.sort_values(by='submission_status', ascending=False).head()

,submission_status
step_id,
31978,16084
32031,13373
32202,13002
33481,10300
32075,10003
